# Báo cáo bài tập lớn cuối kỳ

Họ và tên: Lê Thị Bích Duyên

MSV: 19021254

Lớp: 2122I_INT3405E_20 

## Nội dung
1. Mô tả bài toán
2. Phân tích dữ liệu
3. Tiền xử lý dữ liệu
4. Xây dựng mô hình
5. Huấn luyện và dự đoán
6. Submission

# 1. Mô tả bài toán
Quora là một nền tảng cho phép mọi người có thể đặt câu hỏi và nhận lại những câu trả lời chất lượng từ cộng đồng để giúp đỡ và học hỏi lẫn nhau. 
Bài toán đặt ra là làm sao để loại bỏ những câu hỏi thiếu chân thành - dựa trên những tiên đề sai lầm hoặc có ý định đưa ra một tuyên bố hơn là tìm kiếm những câu trả lời hữu ích.
- Input: câu hỏi trên quora ở dạng text
- Output: 0 hoặc 1 (chân thành hoặc không chân thành)

# 2. Phân tích dữ liệu


In [ ]:
# import các thư viện cần thiết
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
#import os

import nltk
import tensorflow as tf
import tensorflow_addons as tfa
import gc
import re
import spacy

from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from nltk.stem import PorterStemmer, SnowballStemmer
from nltk.stem.lancaster import LancasterStemmer

from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import backend as K
from tensorflow.keras.initializers import Constant
from tensorflow.keras.utils import plot_model
from tensorflow.keras.optimizers import Adam

from sklearn import metrics
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold
from sklearn.datasets import make_classification

from pandarallel import pandarallel
pandarallel.initialize()
from tqdm.notebook import tqdm
from IPython.core.display import display, HTML
tqdm().pandas()


In [ ]:
# đọc tập dữ liệu csv vào dataframe
train_df = pd.read_csv("/kaggle/input/quora-insincere-questions-classification/train.csv")
test_df = pd.read_csv("/kaggle/input/quora-insincere-questions-classification/test.csv")

# train_df.head(5)
train_df.info()
test_df.info()

Dữ liệu train gồm 1306122 hàng với 3 trường dữ liệu
- qid: mã định danh câu hỏi
- question_text: câu hỏi dạng text trên quora
- target: nhãn của câu hỏi, 0 tương ứng với sincere, 1 tương ứng với insincere

Nhận thấy dữ liệu train và test có các giá trị đều hợp lệ, không có giá trị nào null

In [ ]:
# phân tích số lượng câu hỏi sincere và insincere
train_df['words_num'] = train_df.question_text.parallel_apply(lambda x: len(x.split()))

sincere_ques = train_df[train_df.target==0]
insincere_ques = train_df[train_df.target==1]

# xây dựng bảng, biểu đồ phân tích dữ liệu
fig, ax = plt.subplots(1,2)
ax[0] = fig.add_axes([0,0,1,1])
ax[1] = fig.add_axes([1.2,0,1,1])
# bar chart
ax[0].set_title('Statistics of sincere and insincere question')
ax[0].bar(['sincere', 'insincere'], train_df.target.value_counts(),
         color=(0.9, 0.4, 0.6, 0.6))
for p in ax[0].patches:
    height = p.get_height()
    ax[0].annotate('{}'.format(height),
      xy=(p.get_x() + p.get_width() / 2, height),
      xytext=(0, 3), # 3 points vertical offset
      textcoords="offset points",ha='center')
# pie chart
rates = [sincere_ques.shape[0]/train_df.shape[0] * 100, 
        insincere_ques.shape[0]/train_df.shape[0] * 100]

plt.pie(rates, labels=['sincere', 'insincere'],autopct='%1.1f%%', shadow=False, 
        colors=("grey","orange"),startangle=90)
plt.show()

# train_df['target'].value_counts().plot(kind='bar')

In [ ]:
# in ra một số mẫu câu hỏi sincere và insincere
print('sincere question: ')
display(sincere_ques.head(5))
print('insincere question: ')
display(insincere_ques.head(5))

Từ biểu đồ trên ta thấy số câu hỏi sincere chênh lệch khá lớn, gấp khoảng 15 lần so với câu hỏi insincere. Vì kích thước dữ liệu ở các lớp không cân bằng nhau, nên ta sử dụng phép đo Precision-Recall, lấy F1-score (harmonic mean của precision và recall) để đánh giá mô hình. F1 càng cao thì bộ phân lớp càng tốt.

$\frac{2}{F_1}=\frac{1}{precision}+\frac{1}{recall}$

trong đó: 
- precision - độ chính xác: tỉ lệ số điểm true positive (TP) trong số những điểm được phân loại là positive (TP + FP).
- recall - độ bao phủ: tỉ lệ số điểm true positive (TP) trong số những điểm thực sự là positive (TP + FN).

(https://exposedjunction.com/f1-score-la-gi/)

Không thể sử dụng accuracy (tính bằng tỉ lệ số điểm dự đoán đúng/tổng số điểm trong tập kiểm thử) để đánh giá mô hình vì nó sẽ dự đoán câu hỏi hầu hết thuộc nhóm sincere, mô hình sẽ có sai số lớn và không dùng được.


In [ ]:
# phân tích số lượng từ trong câu hỏi
fig2, axes = plt.subplots(1,2)
axes[0] = fig2.add_axes([0,0,1,1.2])
axes[1] = fig2.add_axes([1.2,0,1,1])
# table
rowLabels = ["min_len","mean_len","max_len"]
colLabels = ["sincere_question","insincere_question"]
data = [[np.min(sincere_ques['words_num']), np.min(insincere_ques['words_num'])],
       [np.mean(sincere_ques['words_num']), np.mean(insincere_ques['words_num'])],
       [np.max(sincere_ques['words_num']), np.max(insincere_ques['words_num'])]]
df = pd.DataFrame(data, columns=colLabels)
axes[0].axis('off')
table=axes[0].table(cellText=df.values,colLabels=df.columns,rowLabels=rowLabels,bbox=[0, 0, 1, 0.8])
table.auto_set_font_size(False)
table.set_fontsize(14)
# density plot
axes[1].set_title('Density plot of number of words in every question') 
sns.set(style="darkgrid")
sns.kdeplot(train_df.words_num, shade=True, color='red')
sns.kdeplot(sincere_ques.words_num, shade=True, color='blue')
sns.kdeplot(insincere_ques.words_num, shade=True, color='orange')
plt.legend(["all", "sincere", "insincere"])
plt.show()

Nhận xét: đồ thị của câu hỏi sincere (chân thành) hẹp và dốc hơn câu hỏi không chân thành => câu hỏi chân thành có xu hướng ít từ hơn.
- câu hỏi sincere có độ dài trung bình là 12.5 từ trên một câu, câu dài nhất là 134 từ, hầu hết độ dài đều nằm trong khoảng dưới 55 từ
- câu hỏi insincere có độ dài trung bình 17.3 từ, câu dài nhất là 64 từ và độ dài hầu hết đều nằm trong khoảng 55 từ trở xuống

# 3. Tiền xử lý dữ liệu
Vì dữ liệu dạng text thường phức tạp do bản thân các từ là dữ liệu rời rạc, nhiều nhiễu nên để chuyển thành dạng máy tính có thể hiểu được ta cần phải xử lý trước dữ liệu.

Các bước bao gồm:

**B1: Clean dữ liệu:**
- bỏ các kí tự đặc biệt (dấu câu...)
- bỏ các tag không cần thiết (công thức toán, đường link)
- sửa các lỗi chính tả thường gặp
- lowercase
- chuyển các từ viết tắt về dạng nguyên bản
- bỏ các stopword (ví dụ như the, is,... không có ngữ nghĩa cụ thể) - ở đây em không bỏ các stopword vì tuy các từ này đứng riêng lẻ thì ko có ngữ nghĩa nhưng nếu xóa hết thì nghĩa của câu sẽ bị ảnh hưởng khá nhiều, dự đoán có thể làm giảm độ chính xác của mô hình.

**B2: Tokenization** 

**B3: Convert to sequences** 

In [ ]:
# tạo sample data từ train data để thử nghiệm các hàm tiền xử lý
train_suffle = train_df.sample(frac = 1,random_state=123).reset_index(drop=True)
sample = train_suffle.sample(n=100, random_state=123)

In [ ]:
# từ điển từ viết tắt
contraction_dict= {"cant": 'can not',"doesnt": 'does not',"dont": 'do not',"i'm'a": 'i am about to',"i'm'o": 'i am going to',"i'm": 'i am',"i've": 'i have',"i'll": 'i will',"i'll've": 'i will have',"i'd've": 'i would have',"i'd": 'i would',"whatcha": 'what are you',"amn't": 'am not',"ain't": 'are not',"aren't": 'are not',"'cause": 'because',"can't": 'can not',"can't've": 'can not have',"could've": 'could have',"couldn't": 'could not',"couldn't've": 'could not have',"daren't": 'dare not',"daresn't": 'dare not',"dasn't": 'dare not',"didn't": 'did not','didn’t': 'did not',"don't": 'do not','don’t': 'do not',"doesn't": 'does not',"e'er": 'ever',"everyone's": 'everyone is',"finna": 'fixing to',"gimme": 'give me',"gon't": 'go not',"gonna": 'going to',"gotta": 'got to',"hadn't": 'had not',"hadn't've": 'had not have',"hasn't": 'has not',"haven't": 'have not',"he've": 'he have',"he's": 'he is',"he'll": 'he will',"he'll've": 'he will have',"he'd": 'he would',"he'd've": 'he would have',"here's": 'here is',"how're": 'how are',"how'd": 'how did',"how'd'y": 'how do you',"how's": 'how is',"how'll": 'how will',"isn't": 'is not',"it's": 'it is',"'tis": 'it is',"'twas": 'it was',"it'll": 'it will',"it'll've": 'it will have',"it'd": 'it would',"it'd've": 'it would have',"kinda": 'kind of',"let's": 'let us',"luv": 'love',"ma'am": 'madam',"may've": 'may have',"mayn't": 'may not',"might've": 'might have',"mightn't": 'might not',"mightn't've": 'might not have',"must've": 'must have',"mustn't": 'must not',"mustn't've": 'must not have',"needn't": 'need not',"needn't've": 'need not have',"ne'er": 'never',"o'": 'of',"o'clock": 'of the clock',"ol'": 'old',"oughtn't": 'ought not',"oughtn't've": 'ought not have',"o'er": 'over',"shan't": 'shall not',"sha'n't": 'shall not',"shalln't": 'shall not',"shan't've": 'shall not have',"she's": 'she is',"she'll": 'she will',"she'd": 'she would',"she'd've": 'she would have',"should've": 'should have',"shouldn't": 'should not',"shouldn't've": 'should not have',"so've": 'so have',"so's": 'so is',"somebody's": 'somebody is',"someone's": 'someone is',"something's": 'something is',"sux": 'sucks',"that're": 'that are',"that's": 'that is',"that'll": 'that will',"that'd": 'that would',"that'd've": 'that would have',"em": 'them',"there're": 'there are',"there's": 'there is',"there'll": 'there will',"there'd": 'there would',"there'd've": 'there would have',"these're": 'these are',"they're": 'they are',"they've": 'they have',"they'll": 'they will',"they'll've": 'they will have',"they'd": 'they would',"they'd've": 'they would have',"this's": 'this is',"those're": 'those are',"to've": 'to have',"wanna": 'want to',"wasn't": 'was not',"we're": 'we are',"we've": 'we have',"we'll": 'we will',"we'll've": 'we will have',"we'd": 'we would',"we'd've": 'we would have',"weren't": 'were not',"what're": 'what are',"what'd": 'what did',"what've": 'what have',"what's": 'what is',"what'll": 'what will',"what'll've": 'what will have',"when've": 'when have',"when's": 'when is',"where're": 'where are',"where'd": 'where did',"where've": 'where have',"where's": 'where is',"which's": 'which is',"who're": 'who are',"who've": 'who have',"who's": 'who is',"who'll": 'who will',"who'll've": 'who will have',"who'd": 'who would',"who'd've": 'who would have',"why're": 'why are',"why'd": 'why did',"why've": 'why have',"why's": 'why is',"will've": 'will have',"won't": 'will not',"won't've": 'will not have',"would've": 'would have',"wouldn't": 'would not',"wouldn't've": 'would not have',"y'all": 'you all',"y'all're": 'you all are',"y'all've": 'you all have',"y'all'd": 'you all would',"y'all'd've": 'you all would have',"you're": 'you are',"you've": 'you have',"you'll've": 'you shall have',"you'll": 'you will',"you'd've": 'you would have',"you'd": 'you would','jan.': 'january','feb.': 'february','mar.': 'march','apr.': 'april','jun.': 'june','jul.': 'july','aug.': 'august','sep.': 'september','oct.': 'october','nov.': 'november','dec.': 'december','I’m': 'I am','I’m’a': 'I am about to','I’m’o': 'I am going to','I’ve': 'I have','I’ll': 'I will','I’ll’ve': 'I will have','I’d': 'I would','I’d’ve': 'I would have','amn’t': 'am not','ain’t': 'are not','aren’t': 'are not','’cause': 'because','can’t': 'can not','can’t’ve': 'can not have','could’ve': 'could have','couldn’t': 'could not','couldn’t’ve': 'could not have','daren’t': 'dare not','daresn’t': 'dare not','dasn’t': 'dare not','doesn’t': 'does not','e’er': 'ever','everyone’s': 'everyone is','gon’t': 'go not','hadn’t': 'had not','hadn’t’ve': 'had not have','hasn’t': 'has not','haven’t': 'have not','he’ve': 'he have','he’s': 'he is','he’ll': 'he will','he’ll’ve': 'he will have','he’d': 'he would','he’d’ve': 'he would have','here’s': 'here is','how’re': 'how are','how’d': 'how did','how’d’y': 'how do you','how’s': 'how is','how’ll': 'how will','isn’t': 'is not','it’s': 'it is','’tis': 'it is','’twas': 'it was','it’ll': 'it will','it’ll’ve': 'it will have','it’d': 'it would','it’d’ve': 'it would have','let’s': 'let us','ma’am': 'madam','may’ve': 'may have','mayn’t': 'may not','might’ve': 'might have','mightn’t': 'might not','mightn’t’ve': 'might not have','must’ve': 'must have','mustn’t': 'must not','mustn’t’ve': 'must not have','needn’t': 'need not','needn’t’ve': 'need not have','ne’er': 'never','o’': 'of','o’clock': 'of the clock','ol’': 'old','oughtn’t': 'ought not','oughtn’t’ve': 'ought not have','o’er': 'over','shan’t': 'shall not','sha’n’t': 'shall not','shalln’t': 'shall not','shan’t’ve': 'shall not have','she’s': 'she is','she’ll': 'she will','she’d': 'she would','she’d’ve': 'she would have','should’ve': 'should have','shouldn’t': 'should not','shouldn’t’ve': 'should not have','so’ve': 'so have','so’s': 'so is','somebody’s': 'somebody is','someone’s': 'someone is','something’s': 'something is','that’re': 'that are','that’s': 'that is','that’ll': 'that will','that’d': 'that would','that’d’ve': 'that would have','there’re': 'there are','there’s': 'there is','there’ll': 'there will','there’d': 'there would','there’d’ve': 'there would have','these’re': 'these are','they’re': 'they are','they’ve': 'they have','they’ll': 'they will','they’ll’ve': 'they will have','they’d': 'they would','they’d’ve': 'they would have','this’s': 'this is','those’re': 'those are','to’ve': 'to have','wasn’t': 'was not','we’re': 'we are','we’ve': 'we have','we’ll': 'we will','we’ll’ve': 'we will have','we’d': 'we would','we’d’ve': 'we would have','weren’t': 'were not','what’re': 'what are','what’d': 'what did','what’ve': 'what have','what’s': 'what is','what’ll': 'what will','what’ll’ve': 'what will have','when’ve': 'when have','when’s': 'when is','where’re': 'where are','where’d': 'where did','where’ve': 'where have','where’s': 'where is','which’s': 'which is','who’re': 'who are','who’ve': 'who have','who’s': 'who is','who’ll': 'who will','who’ll’ve': 'who will have','who’d': 'who would','who’d’ve': 'who would have','why’re': 'why are','why’d': 'why did','why’ve': 'why have','why’s': 'why is','will’ve': 'will have','won’t': 'will not','won’t’ve': 'will not have','would’ve': 'would have','wouldn’t': 'would not','wouldn’t’ve': 'would not have','y’all': 'you all','y’all’re': 'you all are','y’all’ve': 'you all have','y’all’d': 'you all would','y’all’d’ve': 'you all would have','you’re': 'you are','you’ve': 'you have','you’ll’ve': 'you shall have','you’ll': 'you will','you’d': 'you would','you’d’ve': 'you would have', 'aka': 'also known as', 'approx.': 'approximately'}
# các lỗi chính tả thường gặp hoặc các từ cách viết khác nhưng cùng nghĩa
mispell_dict = {'colour': 'color', 'centre': 'center', 'favourite': 'favorite', 'travelling': 'traveling', 'counselling': 'counseling', 'theatre': 'theater', 'cancelled': 'canceled', 'labour': 'labor', 'organisation': 'organization', 'wwii': 'world war 2', 'citicise': 'criticize', 'youtu ': 'youtube ', 'Qoura': 'Quora', 'sallary': 'salary', 'Whta': 'What', 'narcisist': 'narcissist', 'howdo': 'how do', 'whatare': 'what are', 'howcan': 'how can', 'howmuch': 'how much', 'howmany': 'how many', 'whydo': 'why do', 'doI': 'do I', 'theBest': 'the best', 'howdoes': 'how does', 'mastrubation': 'masturbation', 'mastrubate': 'masturbate', "mastrubating": 'masturbating', 'pennis': 'penis', 'Etherium': 'bitcoin', 'narcissit': 'narcissist', 'bigdata': 'big data', '2k17': '2017', '2k18': '2018', 'qouta': 'quota', 'exboyfriend': 'ex boyfriend', 'airhostess': 'air hostess', "whst": 'what', 'watsapp': 'whatsapp', 'demonitisation': 'demonetization', 'demonitization': 'demonetization', 'demonetisation': 'demonetization','electroneum':'bitcoin','nanodegree':'degree','hotstar':'star','dream11':'dream','ftre':'fire','tensorflow':'framework','unocoin':'bitcoin','lnmiit':'limit','unacademy':'academy','altcoin':'bitcoin','altcoins':'bitcoin','litecoin':'bitcoin','coinbase':'bitcoin','cryptocurency':'cryptocurrency','simpliv':'simple','quoras':'quora','schizoids':'psychopath','remainers':'remainder','twinflame':'soulmate','quorans':'quora','brexit':'demonetized','iiest':'institute','dceu':'comics','pessat':'exam','uceed':'college','bhakts':'devotee','boruto':'anime','cryptocoin':'bitcoin','blockchains':'blockchain','fiancee':'fiance','redmi':'smartphone','oneplus':'smartphone','qoura':'quora','deepmind':'framework','ryzen':'cpu','whattsapp':'whatsapp','undertale':'adventure','zenfone':'smartphone','cryptocurencies':'cryptocurrencies','koinex':'bitcoin','zebpay':'bitcoin','binance':'bitcoin','whtsapp':'whatsapp','reactjs':'framework','bittrex':'bitcoin','bitconnect':'bitcoin','bitfinex':'bitcoin','yourquote':'your quote','whyis':'why is','jiophone':'smartphone','dogecoin':'bitcoin','onecoin':'bitcoin','poloniex':'bitcoin','7700k':'cpu','angular2':'framework','segwit2x':'bitcoin','hashflare':'bitcoin','940mx':'gpu','openai':'framework','hashflare':'bitcoin','1050ti':'gpu','nearbuy':'near buy','freebitco':'bitcoin','antminer':'bitcoin','filecoin':'bitcoin','whatapp':'whatsapp','empowr':'empower','1080ti':'gpu','crytocurrency':'cryptocurrency','8700k':'cpu','whatsaap':'whatsapp','g4560':'cpu','payymoney':'pay money','fuckboys':'fuck boys','intenship':'internship','zcash':'bitcoin','demonatisation':'demonetization','narcicist':'narcissist','mastuburation':'masturbation','trignometric':'trigonometric','cryptocurreny':'cryptocurrency','howdid':'how did','crytocurrencies':'cryptocurrencies','phycopath':'psychopath','bytecoin':'bitcoin','possesiveness':'possessiveness','scollege':'college','humanties':'humanities','altacoin':'bitcoin','demonitised':'demonetized','brasília':'brazilia','accolite':'accolyte','econimics':'economics','varrier':'warrier','quroa':'quora','statergy':'strategy','langague':'language','splatoon':'game','7600k':'cpu','gate2018':'gate 2018','in2018':'in 2018','narcassist':'narcissist','jiocoin':'bitcoin','hnlu':'hulu','7300hq':'cpu','weatern':'western','interledger':'blockchain','deplation':'deflation', 'cryptocurrencies':'cryptocurrency', 'bitcoin':'blockchain cryptocurrency'}

In [ ]:
# remove tag------------------------------------------------------------------------------------
# xóa các tag không mang nhiều ý nghĩa như url, các biểu thức toán
def remove_tag(x):
    if '[math]' in x:
        x = re.sub('\[math\].*?math\]', 'math_equation', x)
    if 'http' in x or 'www' in x:
        x = re.sub('(?:(?:https?|ftp):\/\/)?[\w/\-?=%.]+\.[\w/\-?=%.]+', 'url', x)
    return x

# remove numbers--------------------------------------------------------------------------------
# thay số bằng # vì hầu hết các embedding đều xử lý text như vậy, sau này khi kết hợp vs embedding
# để huấn luyện mô hình sẽ tốt hơn
def clean_numbers(x):
    if bool(re.search(r'\d', x)):
        x = re.sub('[0-9]{5,}', '#####', x)
        x = re.sub('[0-9]{4}', '####', x)
        x = re.sub('[0-9]{3}', '###', x)
        x = re.sub('[0-9]{2}', '##', x)
    return x

# spell missing--------------------------------------------------------------------------------
def _get_mispell(mispell_dict):
    mispell_re = re.compile('(%s)' % '|'.join(mispell_dict.keys()))
    return mispell_dict, mispell_re

mispellings, mispellings_re = _get_mispell(mispell_dict)
def misspell_fix(text):
    def replace(match):
        return mispellings[match.group(0)]
    return mispellings_re.sub(replace, text)

# abbreviation - từ viết tắt-------------------------------------------------------------------
def contraction_fix(word):
    try:
        a=contraction_dict[word]
    except KeyError:
        a=word
    return a

def abbreviation_fix(text):
    text = " ".join([contraction_fix(w) for w in text.split()])
    return text

#stemming text--------------------------------------------------------------------------------
# ex: studies --> studi, fishes-->fish
def stemming(text):
    ls = LancasterStemmer()
    text = " ".join([ls.stem(w) for w in text.split()])
    return text

# example-----------------------------------
text = "i am travelling around the world."
text2 = "i'd've gone to japan"
print('before\n'+text+'\n'+text2)
print('after')
print(misspell_fix(text))
print(stemming(text))
print(abbreviation_fix(text2))

In [ ]:
# sử dụng các hàm bên trên để clean text
# clean text gồm: xóa tag, sửa lỗi misspell, từ viết tắt,---------------------------------------------
                # xóa kí tự đặc biệt, clean số
def clean_text(text, lowercase=True):
    text = remove_tag(text)
    if lowercase:
        text = text.lower()
    text = misspell_fix(text)
    text = abbreviation_fix(text)
    # xóa kí tự đặc biệt khác chữ và số
    text = re.sub(r'[^a-zA-Z0-9]', ' ', text)
    # clean số
    text = clean_numbers(text)
    # text = stemming(text)
    return text

# clean text cho toàn bộ tập dữ liệu------------------------------------------------------------------
def clean_data(question_text, lowercase=True, preview=False):
    data_clean = pd.DataFrame()
    data_clean['question_text'] = question_text
    data_clean['clean']= data_clean.question_text.parallel_apply(lambda x: clean_text(x, lowercase))
    if preview:
        display(data_clean.head(100))
    return data_clean.clean

# example---------------------------------------------------------
text='I really like may'
print(text.lower())
sample['clean']=clean_data(sample["question_text"], preview=True)
#display(sample)


### Tokenization và Convert to sequences:
- `create_tokenizer()`: để chuyển 1 câu thành tập các từ riêng lẻ, (mỗi token là một từ), sử dụng Tokenizer của keras để tạo một bộ từ điển token, từ xuất hiện càng nhiều thì index của nó càng nhỏ
    - khi sử dụng bộ vocab tokenizer được tạo ra để chuyển văn bản thành chuỗi số thì có thể có những từ ở văn bản không nằm trong tokenizer, vì vậy ta dùng `'<OOV>'` để thay thế cho chúng.
    VD: tokenizer tìm được: `['<OOV>':1,'this': 2, 'fat': 3]`
    
    văn bản cần chuyển: `text` = `this cat is very fat`
    
    => sau khi sử dụng tokenizer: `text` = `[2 1 1 1 3]`
- `convert_to_sequences()`: sử dụng từ điển đã tạo để chuyển text sang chuỗi số để máy tính hiểu được, padding và truncating để đồng bộ chiều dài các câu thành một giá trị cố định

    dựa vào biểu đồ đã phân tích ở phần 2 ta nhận thấy hầu hết các câu hỏi đều có độ dài từ 55 từ trở xuống, câu dài hơn có số lượng rất ít nên ta chọn độ dài chung cho các chuỗi là 70 từ


In [ ]:
SEQUENCES_LENGTH = 70

# tokenize text------------------------------------------------------------------------------------
def create_tokenizer(text_data, preview=False, amount=10):
    tokenizer = Tokenizer(oov_token='<OOV>')
    tokenizer.fit_on_texts(list(text_data))    
    if preview:
        print("Size of Vocab dictionary: ", len(tokenizer.word_index))
        print("Preview: "+str(amount))
        print(list(tokenizer.word_index.items())[:amount])
    return tokenizer

# convert text to sequence by token dictionary----------------------------------------------------
def convert_to_sequences(tokenizer, text_data, padding=True, sequences_len=0):
    # convert to sequences
    word_sequences = tokenizer.texts_to_sequences(text_data)
    # padding sequences để các chuỗi đều có độ dài bằng nhau
    if padding:
        word_sequences= pad_sequences(word_sequences, maxlen=sequences_len, 
                          padding='post', truncating='post') 
    return word_sequences  
    
# example----------------------------------------------
tokenizer=create_tokenizer(sample.clean, preview=True)
#print(len(token_data.word_index)+1)
padded_sequences = convert_to_sequences(tokenizer, sample['clean'],
                                        sequences_len=SEQUENCES_LENGTH)
print("5 first sequences:")
for sequence in padded_sequences[:5]:
    print(sequence)

Như vậy là các hàm preprocessing đã xây dựng và hoạt động tốt, em sẽ áp dụng để xử lý dữ liệu của bài toán ở phần 5.

# 4. Mô hình huấn luyện

Xây dựng mô hình huấn luyện bằng Bidirectional LSTM (Long Short-Term Memory Networks)

Trước khi hiểu mô hình LSTM là gì thì ta cần hiểu qua về mạng RNN - recurrent neural network.
RNN có thể mang thông tin từ các layer trước đến layer sau, nên nó có thể dùng để xử lý thông tin dạng chuỗi. Một ví dụ của RNN trong bài toán dự đoán video, RNN có thể mang thông tin của frame ảnh từ state trước tới state sau, tuy nhiên state ở trước đó càng xa thì càng bị vanishing gradient, nghĩa là thông tin chỉ mang được qua một lượng state nhất định
hay nói cách khác là model chỉ học được từ các state gần nó - short term memory

Vì vậy, mô hình Long short term memory ra đời để giải quyết vấn đề khi ta cần các thông tin từ state ở trước đó rất xa và tránh được vanishing gradient. Nó vẫn giữ tư tưởng chính của RNN là sự sao chép kiến trúc theo dạng chuỗi nhưng có phần phức tạp hơn.

- Bidirectional LSTM là một biến thể của LSTM. Trong khi LSTM đơn hướng do đầu vào duy nhất mà nó lấy là từ quá khứ nên nó không thể lưu giữ các thông tin ở tương lai. BiLSTM sẽ chạy thông tin đầu vào theo hai cách: từ quá khứ đến tương lai và từ tương lai trở lại. Do đó nó có thể hiểu được ngữ cảnh của văn bản tốt hơn.

VD: một ví dụ về dự đoán cần thông tin từ tương lai trong đoạn text là:

   `Tôi phải đi học. Hôm qua trời ... Sáng nay đường ướt và trơn trượt`

Ta không thể đoán được nếu như chỉ sử dụng thông tin từ quá khứ, mà cần sử dụng thông tin ở câu sau, tức là từ tương lai ` sáng nay` để dự đoán từ còn thiếu là `mưa` 



In [ ]:
# f1 score
f1 = tfa.metrics.F1Score(num_classes=1, threshold=0.5)
print('success')
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, 
                              patience=2, min_lr=0.0001, verbose=0)
print('success')
earlystopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0.0001, patience=5, 
                              verbose=1, mode='auto', restore_best_weights=True)
print('success')
callbacks = [earlystopping, reduce_lr]

# hàm để xây dựng model-------------------------------------------------------------------------------
def build_Model_LSTM(features=120000, embedding_dim=200, lr=0.001, embedding_matrix=None):
    input_x = Input(shape=(SEQUENCES_LENGTH))
    if not(embedding_matrix is None):
        embedding = Embedding(features, embedding_dim, input_length=SEQUENCES_LENGTH,
                              weights=[embedding_matrix], trainable=False)(input_x)
    else:
        embedding = Embedding(features, embedding_dim, input_length=SEQUENCES_LENGTH)(input_x)
    x = Bidirectional(LSTM(128, return_sequences=True))(embedding)
    x = Bidirectional(LSTM(64, return_sequences=True))(x)
    x = GlobalMaxPool1D()(x)
    x = Dense(64, activation="relu")(x)
    x = Dense(1, activation="sigmoid")(x)
    model = Model(inputs=input_x, outputs=x)
    opt = Adam(lr)
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=[f1])
    return model

# 5. Huấn luyện và dự đoán
## 5.1. Huấn luyện

In [ ]:
EMBEDDING_DIM = 300

batch_size = 512
num_epochs = 10
learning_rate = 0.001


# B1: preprocessing=====================================================================
# clean tập dữ liệu train và dữ liệu test
print('B1: clean text: wait about 40s...')
train_text = clean_data(train_suffle.question_text, lowercase=True)
train_target = train_suffle.target

test_X = clean_data(test_df.question_text, lowercase=True)

# B2: chia train_df thành 2 tập train và validate theo tỉ lệ 8:2========================
print('B2: splitting train_df')
train_X, val_X, train_Y, val_Y = train_test_split(train_text, train_target,
                                                  test_size=0.2, random_state=165)

# B3: convert to sequences==============================================================
print('B3: convert to sequences....')

tokenizer = create_tokenizer(train_X)
print('Create vocab dict success. Start converting..')

train_X_seq = convert_to_sequences(tokenizer, train_X, sequences_len=SEQUENCES_LENGTH)
val_X_seq = convert_to_sequences(tokenizer, val_X, sequences_len=SEQUENCES_LENGTH)
test_X_seq = convert_to_sequences(tokenizer, test_X, sequences_len=SEQUENCES_LENGTH)
print('Convert successfully.')

features = len(tokenizer.word_index)+1
print('features: '+str(features))

Sử dụng TPU để cải thiện tốc độ training, em thấy nhanh hơn khoảng 3 lần so với GPU và không gặp tình trạng bị dừng lại do truy cập quá mức bộ nhớ

https://www.kaggle.com/kutaykutlu/tpus-resnet50-leaf-disease#Detect-TPU

In [ ]:
# sử dụng TPU
strategy = None
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect() # TPU detection
    strategy = tf.distribute.TPUStrategy(tpu)
    print('TPU')
except ValueError:
    if len(tf.config.list_physical_devices('GPU')) > 0:
        strategy = tf.distribute.MirroredStrategy()
        print('GPU')
    else:
        strategy = tf.distribute.get_strategy()
        print('CPU')

Khi không sử dụng thêm các pretrain embedding thì private score đạt được là 0.644 có thể xem ở version 4 và đoạn code dưới đây.

In [ ]:
with strategy.scope():
    # B4: build model=====================================================================
    f1 = tfa.metrics.F1Score(num_classes=1, threshold=0.5)
    print('B4: bulding model....')
    model = build_Model_LSTM()
    model.summary()

#     # B5: train model=====================================================================
#     print('B5: start tranning....')
#     model.fit(train_X_seq, train_Y, batch_size=batch_size, epochs=num_epochs, 
#               validation_data=(val_X_seq, val_Y), callbacks=callbacks)

## 5.2. Sử dụng model đã huấn luyện để dự đoán

In [ ]:
# tìm threshold phù hợp để đạt được f1_score cao nhất
def best_threshold(y_train,train_preds):
    f1_score = [0,0,0] # idx, cur, max
    threshold = 0
    for f1_score[0] in tqdm(np.arange(0.1, 0.9, 0.01)):
        f1_score[1] = metrics.f1_score(y_train, np.array(train_preds)>f1_score[0])
        if f1_score[1] > f1_score[2]:
            threshold = f1_score[0]
            f1_score[2] = f1_score[1]
    print('best threshold: ')
    print(threshold)
    print(f1_score[2])
    return threshold, f1_score[2]

In [ ]:
def predict_by(trained_model):
    val_pred = trained_model.predict(val_X_seq, verbose=1, batch_size=256)
    threshold, f1_score = best_threshold(val_Y, val_pred)

    test_pred = trained_model.predict(test_X_seq, verbose=1, batch_size=256)

    print(metrics.classification_report(val_Y,(val_pred>threshold).astype(int)))
    return threshold, f1_score, val_pred, test_pred

# predict by trained model========================================================
# with strategy.scope():
#     test_pred, threshold = predict_by(model)

### Cải thiện dự đoán bằng cách kết hợp các embedding cho sẵn
Ở đây em thử nghiệm với 3 loại embedding là:
- Paragram
- GloVe - Global Vectors: mô hình là thuật toán học tập không giám sát để tạo ra các biểu diễn vector cho các từ, GloVe học bằng cách xây dựng ma trận đồng xuất hiện, cơ bản là nó tính tần suất một từ xuất hiện trong ngữ cảnh
- Wiki: hay FastText khá khác với nhúng trên, trong khi GloVe coi đơn vị nhỏ nhất để đào tạo là một từ thì FastText sử dụng các kí tự n-gram làm đơn vị huấn luyện nhỏ nhất. Lợi ích lớn nhất của việc sử dụng FastText là nó tạo ra các nhúng từ tốt hơn cho các từ hiếm hoặc thậm chí là các từ không được tìm ra trong quá trình trainning do các vector kí tự n-gram có khả năng xuất hiện trong các từ khác nhiều hơn. Đây là điều GloVe không thể đạt được.
    
    VD: word vector `apple` có thể được chia thành các word vector unit riêng biệt là `ap`, `app`, `ple`.


In [ ]:
def get_coefs(word, *arr): 
    return word, np.asarray(arr, dtype='float32')
# lấy số dòng của file embedding
def get_lines_num(file_name): 
    return sum(1 for _ in open(file_name, encoding="utf8", errors='ignore'))
# chuyển file embedding thành dict
def load_embedding(file_name): 
    print('loading embedding file..')
    return dict(get_coefs(*o.split(" ")) for o in tqdm(open(file_name, encoding="utf8", errors='ignore'), 
                                                       total=get_lines_num(file_name)) if len(o) > 100)

def embedding_matrix(embeddings_vec, features, token_word_index):
    print('load success. start embedding matrix..')
    # features = len(tokenizer.word_index)+1
    # embedding_vec is dict created by load_embedding()
    all_embs = np.stack(embeddings_vec.values()) 
    emb_mean,emb_std = all_embs.mean(), all_embs.std()
    embed_size = all_embs.shape[1]

    nb_words = features
    embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, embed_size))
    print('yes1')
    for word, i in token_word_index.items():
        if i >= features: continue
        embedding_vector = embeddings_vec.get(word) 
        if embedding_vector is not None: embedding_matrix[i] = embedding_vector
    print('yes2')
    return embedding_matrix

In [ ]:
GLOVE_PATH = 'glove.840B.300d/glove.840B.300d.txt'
PARA_PATH = 'paragram_300_sl999/paragram_300_sl999.txt'
WIKI_PATH = 'wiki-news-300d-1M/wiki-news-300d-1M.vec'

!unzip -n /kaggle/input/quora-insincere-questions-classification/embeddings.zip {GLOVE_PATH} -d .

glove_vec = load_embedding(GLOVE_PATH)

### a. Huấn luyện model với GloVe embedding
Kết quả đạt được là f1_score đã tăng lên 0.68 so với f1_score khi không sử dụng nhúng là 0.64. Khi huấn luyện có thêm GloVe ở version 8 thì Private score tăng lên 0.686 

In [ ]:
glove_emb_matrix = embedding_matrix(glove_vec, features, tokenizer.word_index)

In [ ]:
# apply glove
# TPU
with strategy.scope():
# B4: build model========================================================================
    f1 = tfa.metrics.F1Score(num_classes=1, threshold=0.5)
    print('B4: bulding model with glove embedding....')
    glove_model = build_Model_LSTM(features, EMBEDDING_DIM, learning_rate, glove_emb_matrix)
    #glove_model.summary()

# B5: train model========================================================================
    print('B5: start tranning....')
    glove_model.fit(train_X_seq, train_Y, batch_size=batch_size, epochs=num_epochs, 
              validation_data=(val_X_seq, val_Y), callbacks=callbacks)
# predict by trained model===============================================================
    glove_threshold, glove_f1_score, glove_val_pred, glove_test_pred = predict_by(glove_model)
    
    

In [ ]:
# giảm tải cho bộ nhớ
del glove_vec
gc.collect()

### b. Huấn luyện model với wiki

In [ ]:
!unzip -n /kaggle/input/quora-insincere-questions-classification/embeddings.zip {WIKI_PATH} -d .

wiki_vec = load_embedding(WIKI_PATH)

In [ ]:
wiki_emb_matrix = embedding_matrix(wiki_vec, features, tokenizer.word_index)

In [ ]:
# apply wiki
# TPU
with strategy.scope():
# B4: build model========================================================================
    f1 = tfa.metrics.F1Score(num_classes=1, threshold=0.5)
    print('B4: bulding model with glove embedding....')
    wiki_model = build_Model_LSTM(features, EMBEDDING_DIM, learning_rate, wiki_emb_matrix)

# B5: train model========================================================================
    print('B5: start tranning....')
    wiki_model.fit(train_X_seq, train_Y, batch_size=batch_size, epochs=num_epochs, 
              validation_data=(val_X_seq, val_Y), callbacks=callbacks)
# predict by trained model===============================================================
    wiki_threshold, wiki_f1_score, wiki_val_pred, wiki_test_pred = predict_by(wiki_model)

In [ ]:
# giảm tải cho bộ nhớ
del wiki_vec
gc.collect()

### c. Huấn luyện model với para

In [ ]:
!unzip -n /kaggle/input/quora-insincere-questions-classification/embeddings.zip {PARA_PATH} -d .

para_vec = load_embedding(PARA_PATH)

In [ ]:
para_emb_matrix = embedding_matrix(para_vec, features, tokenizer.word_index)

In [ ]:
# apply para
# TPU
with strategy.scope():
# B4: build model========================================================================
    f1 = tfa.metrics.F1Score(num_classes=1, threshold=0.5)
    print('B4: bulding model with glove embedding....')
    para_model = build_Model_LSTM(features, EMBEDDING_DIM, learning_rate, para_emb_matrix)

# B5: train model========================================================================
    print('B5: start tranning....')
    para_model.fit(train_X_seq, train_Y, batch_size=batch_size, epochs=num_epochs, 
              validation_data=(val_X_seq, val_Y), callbacks=callbacks)
# predict by trained model===============================================================
    para_threshold, para_f1_score, para_val_pred, para_test_pred = predict_by(para_model)

In [ ]:
# giảm tải cho bộ nhớ
del para_vec
gc.collect()

# 6. Submission
Kết hợp các kết quả từ 3 mô hình. f1_score ở cả 3 không có sự chênh lệch quá lớn và đều cải thiện độ chính xác hơn so với không sử dụng nên ta lấy trung bình 3 kết quả.

In [ ]:
val = np.zeros((len(val_X),), dtype=np.float32)

val += 1/3 * np.squeeze(glove_val_pred)
val += 1/3 * np.squeeze(para_val_pred)
val += 1/3 *np.squeeze(wiki_val_pred)
threshold_global, f1_global = best_threshold(val_Y, val)
print(metrics.classification_report(val_Y,(val>threshold_global).astype(int)))

pred = np.zeros((len(test_X),), dtype=np.float32)
pred += 1/3 * np.squeeze(glove_test_pred)
pred += 1/3 * np.squeeze(para_test_pred)
pred += 1/3 * np.squeeze(wiki_test_pred)
test_pred=((pred>threshold_global).astype(int))


Sau khi kết hợp cả 3 embedding ta thấy f1-score có tăng lên một chút so với chạy riêng lẻ với từng embedding.

In [ ]:
#test_pred = (test_pred>threshold).astype(int)

submit = pd.DataFrame({"qid":test_df["qid"].values})
submit['prediction'] = test_pred
submit.to_csv("submission.csv", index=False)

# Reference
- clean data: https://www.exxactcorp.com/blog/Deep-Learning/text-preprocessing-methods-for-deep-learning
- model with embeddings: https://www.kaggle.com/sbongo/do-pretrained-embeddings-give-you-the-extra-edge